# Итераторы и генераторы

### Итератор - объект-перечислитель, который "знает" о следующем элементе. 

#### Допустим, есть список x = [1, 2]. У списка есть итератор, который при первом запросе ( функия next() ) вернет 1, при втором - 2, а при третьем запросе вызовет ошибку StopIteration.

In [2]:
x = [1, 2]
iterator = iter(x)
print(next(iterator))
print(next(iterator))
print(next(iterator))

1
2


StopIteration: 

#### Как на самом деле работает цикл for:

In [5]:
# Пример со словарем
book = {
    'title': 'The Langoliers',
    'author': 'Stephen King',
    'year_published': 1990
}

# Стандартный цикл for
for i in book:
    print(i, end = ' ')

print()
    
# Альтернатива циклу for
it = iter(book)
while True:
    try:
        i = next(it)
        print(i, end = ' ')
    except StopIteration:
        break
        
# Результат работы будет идентичен для любого итерируемого множества значение (словарь, список и т.д.)

year_published author title 
year_published author title 

#### Итераторы удобно использовать для перебора значений в собственном классе

##### Для того, чтобы сделать класс итерируемым, необходимо добавить в него метод __next__

In [32]:
from random import random

# Итератор данного класса возвращает число от 0 до 1
class RandomIterator:
    def __init__(self, k):
        self.k = k # Количество чисел, которые будут выведен
        self.i = 0 # Количество уже перечисленных чисел
        
    def __next__(self):
        if self.i < self.k:
            self.i += 1
            return random()
        else:
            raise StopIteration
        
x = RandomIterator(3) # Создаем класс для вывода 3 случайных чисел
print(next(x)) # next(x) ~ x.__next__(), x -- iterator
print(next(x))
print(next(x))
print(next(x)) # При попытке вывести 4-ое число, возникает ошибка StopItaration

0.6031746933791006
0.9320624366212921
0.3720349275674475


StopIteration: 

##### Для того, чтобы пользоваться собственным интеретором в цикле for, необходимо сделать так, чтобы к нашему класса стала применима функция iter(OurClass) 

##### Для этого в классе добавляется новый метод - __iter__(self)

In [33]:
from random import random

# Итератор данного класса возвращает число от 0 до 1
class RandomIterator:
    def __iter__(self):
        return self
    
    def __init__(self, k):
        self.k = k # Количество чисел, которые будут выведен
        self.i = 0 # Количество уже перечисленных чисел
        
    def __next__(self):
        if self.i < self.k:
            self.i += 1
            return random()
        else:
            raise StopIteration
            
for x in RandomIterator(10):
    print(x)

0.7511639708765198
0.6725760817590697
0.5242811862821755
0.13444313889360582
0.26930906566637225
0.8388957781049017
0.043991401100652805
0.7284012259893162
0.7424071456845994
0.3444885809651007


##### Пример итератора для вовзращения пары значений

In [37]:
class DoubleElementListIterator:
    def __init__(self, lst):
        self.lst = lst # Сам список
        self.i = 0 # Отсчет 
        
    def __next__(self):
        if self.i < len(self.lst): # если i < размера списка
            self.i += 2 # прибавляем 2 к i
            return self.lst[self.i - 2], self.lst[self.i - 1] # и возвращаем два значения из списка
        else:
            raise StopIteration
            
class MyList(list): # Класс MyList наследуется от класса list
    def __iter__(self):
        return DoubleElementListIterator(self) # Почему объект DoubleElementListIterator возвращаем два элемента ?
    
for pair in MyList([1, 2, 3, 4]):
    print(pair)


(1, 2)
(3, 4)


##### Для таких простых задач использование итераторов накладно, т.к. надо создавать классы, а это не быстро и занимает много строчек кода. Как раз для это в python есть генераторы.

### Генератор - функция, которая вместо того, чтобы возвращать какое-нибудь значение, генерирует его.

##### Для этого используется ключевое слова yield

In [6]:
def random_generator(k):
    for i in range(k):
        yield random()
        
gen = random_generator(3)
print(type(gen))

<class 'generator'>


##### Работа генератора заключается в следующем: интерпретатор выполняет тело генератора до тех пор пока не встретит ключевое слово yield, после чего вернет значение. Однако, при следующем вызове генератора, интерпретатор продолжет выполнение с того места, на котором он остановился и до нового значения yield. И так далее.

##### Если после очередного вызова интерпретатор дойдет до конца генератора и не встретит ключевое слово yield, то будет вызвана ошибка StopIteration.

##### Пример

In [10]:
def simple_gen():
    print("Checkpoint 1")
    yield 1
    print("Checkpoint 2")
    yield 2
    print("Checkpoint 3")
    
gen = simple_gen()
x = next(gen)
print(x)
y = next(gen)
print(y)
z = next(gen)


Checkpoint 1
1
Checkpoint 2
2
Checkpoint 3


StopIteration: 

In [11]:
for i in simple_gen():
    print(i)

Checkpoint 1
1
Checkpoint 2
2
Checkpoint 3


##### Такой подход в программировании носит название "Концепция отложенного исполнения"

##### Таким образом удобный генератор для k случайных чисел будет выглядеть следующим образом

In [24]:
from random import random

def random_generator(k):
    for i in range(k):
        yield random()

gen = random_generator(3)
for i in gen:
    print(i)

0.9459063795938274
0.7598931795804734
0.6949079834391128


##### Если внутри генератора интерпретатор встретит return, то будет вызвана ошибка StopItaration, а возвращаемое значение return будет сообщением об ошибке.

In [25]:
def simple_gen():
    print("Checkpoint 1")
    yield 1
    print("Checkpoint 2")
    yield 2
    print("Checkpoint 3")
    return 'No more elements'

gen = simple_gen()
print(next(gen))
print(next(gen))
print(next(gen))


Checkpoint 1
1
Checkpoint 2
2
Checkpoint 3


StopIteration: No more elements

###### Сообщение No more elements было взято из return

###### Задача 1

In [ ]:
class multifilter:
    def judge_half(pos, neg):
        # допускает элемент, если его допускает хотя бы половина фукнций (pos >= neg)
        return pos >= neg

    def judge_any(pos, neg):
        # допускает элемент, если его допускает хотя бы одна функция (pos >= 1)
        return pos >= 1

    def judge_all(pos, neg):
        # допускает элемент, если его допускают все функции (neg == 0)
        return neg == 0

    def __init__(self, iterable, *funcs, judge=judge_any):
        # iterable - исходная последовательность
        # funcs - допускающие функции
        # judge - решающая функция
        self.pos = 0;
        self.neg = 0;
        self.i = 0
        self.n = len(iterable)
        self.iterable = iterable
        self.funcs = funcs
        self.judge = judge

    def __next__(self):
        while True:
            if self.i < self.n:
                for f in self.funcs:
                    if f(self.iterable[self.i]):
                        self.pos += 1
                    else:
                        self.neg += 1

                if self.judge(self.pos, self.neg):
                    self.pos = 0
                    self.neg = 0
                    res = self.i
                    self.i += 1
                    return res
                self.i += 1
                self.pos = 0
                self.neg = 0
            else:
                raise StopIteration

    def __iter__(self):
        # возвращает итератор по результирующей последовательности
        return self

##### Задача 2

In [ ]:
def primes():
    prime = True
    i = 1
    while True:
        i += 1
        for j in range(2, i):
            if i % j == 0:
                prime = False
        if prime:
            yield i
        else:
            prime = True